In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import MSO_AUTO_SIZE
from pptx.dml.color import RGBColor
import logging
import math

logger = logging.getLogger(__name__)

class AdvancedSlideManager:
    def __init__(self, template_path):
        self.prs = Presentation(template_path)
        self._validate_template()
        
        # Template metrics from user verification
        self.ROWS_PER_TEXTBOX = 16
        self.CHARS_PER_ROW = 31
        self.LINE_HEIGHT = Inches(5/16)  # 5" height / 16 rows
        
        # Layout tracking
        self.current_slide = None
        self.current_textbox = None
        self.slide_count = 0
        self.textbox_count = 0
        
        # Initialize first slide
        self._add_new_slide()

    def _validate_template(self):
        """Ensure template has required slides"""
        if len(self.prs.slides) < 1:
            raise ValueError("Template must contain at least one slide")

    def _add_new_slide(self):
        """Add new dual-column slide"""
        self.current_slide = self.prs.slides.add_slide(self.prs.slide_layouts[6])
        
        # Left column (4" width, 5" height)
        left = self.current_slide.shapes.add_textbox(Inches(0.5), Inches(1), Inches(4), Inches(5))
        left.name = "textMainBullets_L"
        left.text_frame.word_wrap = True
        left.text_frame.auto_size = MSO_AUTO_SIZE.NONE
        
        # Right column (4" width, 5" height)
        right = self.current_slide.shapes.add_textbox(Inches(4.7), Inches(1), Inches(4), Inches(5))
        right.name = "textMainBullets_R"
        right.text_frame.word_wrap = True
        right.text_frame.auto_size = MSO_AUTO_SIZE.NONE
        
        self.slide_count += 1
        self.current_textbox = left
        self.textbox_count = 0  # Reset column counter for new slide

    def _split_md_to_chunks(self, md_content):
        """Split markdown into textbox-sized chunks (16 rows x 31 chars)"""
        chunks = []
        current_chunk = []
        current_line = ""
        
        for line in md_content.split('\n'):
            line = line.strip()
            if not line:
                continue
                
            # Split line into 31-character segments
            words = line.split()
            for word in words:
                if len(current_line) + len(word) + 1 > self.CHARS_PER_ROW:
                    current_chunk.append(current_line.strip())
                    current_line = ""
                    if len(current_chunk) >= self.ROWS_PER_TEXTBOX:
                        chunks.append(current_chunk)
                        current_chunk = []
                current_line += f"{word} "
            
            if current_line:
                current_chunk.append(current_line.strip())
                current_line = ""
            
            if len(current_chunk) >= self.ROWS_PER_TEXTBOX:
                chunks.append(current_chunk)
                current_chunk = []
        
        if current_chunk:
            chunks.append(current_chunk)
        
        return chunks

    def _advance_container(self):
        """Move to next column or create new slide"""
        self.textbox_count += 1
        if self.textbox_count % 2 == 0:
            self._add_new_slide()
        else:
            self.current_textbox = self.current_slide.shapes[-1]  # Right column

    def _fill_textbox(self, chunk):
        """Fill current textbox with formatted content"""
        tf = self.current_textbox.text_frame
        tf.clear()
        
        for line in chunk:
            p = tf.add_paragraph()
            p.text = line
            p.font.size = Pt(14)
            
            # Header formatting
            if line.startswith('##'):
                p.font.bold = True
                p.font.size = Pt(18)
            elif line.startswith('###'):
                p.font.italic = True
                p.font.size = Pt(16)

    def process_md(self, md_content):
        """Process markdown with precise layout control"""
        chunks = self._split_md_to_chunks(md_content)
        logger.info(f"Content requires {len(chunks)} textboxes across {math.ceil(len(chunks)/2)} slides")
        
        for chunk in chunks:
            self._fill_textbox(chunk)
            self._advance_container()

    def save(self, output_path):
        """Save final presentation"""
        self.prs.save(output_path)
        logger.info(f"Saved presentation with {self.slide_count} slides")


# Usage example
if __name__ == "__main__":
    import sys
    if len(sys.argv) != 3:
        print("Usage: python report_generator.py input.md output.pptx")
        sys.exit(1)
    
    with open(sys.argv[1], 'r') as f:
        md_content = f.read()
    
    manager = AdvancedSlideManager("fixed_template.pptx")
    manager.process_md(md_content)
    manager.save(sys.argv[2])


In [14]:
def create_optimized_template():
    prs = Presentation()
    
    # Single-column slide (Title content)
    slide1 = prs.slides.add_slide(prs.slide_layouts[6])
    title_box = slide1.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(9), Inches(1.5))
    title_box.name = "title_box"
    title_box.text_frame.word_wrap = False
    title_box.text_frame.auto_size = MSO_AUTO_SIZE.NONE

    # Dual-column slides (Main content)
    for _ in range(2):  # Create 2 master slides
        slide = prs.slides.add_slide(prs.slide_layouts[6])
        for idx, col in enumerate(['L', 'R']):
            left = Inches(0.5) if col == 'L' else Inches(4.7)
            textbox = slide.shapes.add_textbox(left, Inches(1.2), Inches(4), Inches(5))
            textbox.name = f"content_{col}"
            tf = textbox.text_frame
            tf.auto_size = MSO_AUTO_SIZE.NONE
            tf.word_wrap = True
            tf.vertical_anchor = MSO_VERTICAL_ANCHOR.TOP
            tf.margin_left = Inches(0.1)
            tf.margin_right = Inches(0.1)
    
    prs.save("fixed_template.pptx")


In [15]:
md_content = """
## Current Assets
### Cash and Cash Equivalents
Currency in checking/savings accounts
Short-term Treasury bills (maturing <3 months)
Commercial paper from AAA-rated corporations
Money market funds with daily liquidity
Petty cash reserves for office expenses
Foreign currency holdings in major currencies
Undeposited checks from customers
Cash in transit between bank accounts
### Marketable Securities
Corporate bonds with <1yr maturity
Government agency securities
Certificates of deposit (CDs)
Bankers' acceptances
Commercial paper holdings
Treasury notes maturing within 12 months
Highly liquid ETF positions
### Accounts Receivable
Trade receivables from normal operations
Installment receivables from long-term contracts
Receivables from affiliated companies
Allowance for doubtful accounts calculation
Aging schedule analysis (30/60/90 days)
Credit memo adjustments
Factored receivables disclosure
Unbilled receivables from progress contracts

## Non-Current Assets
### Property, Plant & Equipment
Land acquisition costs (original purchase)
Building improvements capitalization
Machinery installation costs
Equipment depreciation schedules
Leasehold improvement amortization
Construction-in-progress accounts
Capitalized interest during construction
### Intangible Assets
Patent acquisition and amortization
Trademark registration/maintenance costs
Customer list valuations
Non-compete agreement valuations
Software development costs
Licensing agreements fair value
Goodwill impairment testing methodology
### Long-Term Investments
Held-to-maturity securities portfolio
Equity method investment accounting
Real estate held for appreciation
Venture capital fund investments
Convertible debt instruments
Restricted stock holdings
Investments in subsidiaries

## Current Liabilities
### Accounts Payable
Trade payables to suppliers
Accrued purchases for goods received
Third-party processor withholdings
Construction retainage payable
Dividends declared but unpaid
Customer deposits/advance payments
Escheat liability estimates
### Short-Term Debt
Commercial paper outstanding
Revolving credit facility draws
Current portion of long-term debt
Bank overdraft facilities used
Short-term lease liabilities
Vendor financing arrangements
Convertible debt equity component

## Long-Term Liabilities
### Bonds Payable
Corporate bond issuance at premium/discount
Debenture conversion features
Sinking fund requirements
Unamortized bond issuance costs
Fair value hedge adjustments
Callable bond provisions
Convertible bond accounting
### Pension Liabilities
Defined benefit obligation calculations
Actuarial gains/losses recognition
Plan asset valuations
Curtailment/settlement accounting
Multi-employer plan disclosures
Post-employment benefits accrual
Termination benefit provisions

## Shareholders' Equity
### Common Stock
Par value per share disclosure
Authorized shares vs outstanding
Treasury stock accounting method
Stock split adjustments
Stock option pool reserves
Restricted stock unit accruals
Dividend reinvestment plan shares
### Retained Earnings
Prior period adjustments
Dividend declaration accounting
ESOP allocation impacts
Foreign currency translation adjustments
Hedging reserve balances
Revaluation surplus accounts
Accumulated other comprehensive income

"""

In [16]:
manager = AdvancedSlideManager("fixed_template.pptx")
manager.process_md(md_content)
manager.save("financial_report.pptx")